# General setup

In [5]:


%run BREPpy.py

#conn = Connector()
#conn.init_from_script(['--config_fn','./input_files/Parameters.hoc',
#                       '--goc_points_fn', './example_simulation/coordinates_input/GoCcoordinates.dat',
#                      '--gc_points_fn', './example_simulation/coordinates_input/GCcoordinates.dat'])


p1 = './example_simulation/coordinates_input/'
p2 = './example_simulation/coordinates_input/subsampled/'

go_ori = p1+'GoCcoordinates.dat'
gr_ori = p1+'GCcoordinates.dat'
go_32 = p2+'GoCcoordinates_32.dat'
gr_32 = p2+'GCcoordinates_32.dat'
go_16 = p2+'GoCcoordinates_16.dat'
gr_16 = p2+'GCcoordinates_16.dat'
go_4 = p2+'GoCcoordinates_4.dat'
gr_4 = p2+'GCcoordinates_4.dat' 

# read in parameter file
empty_hoc = dir(neuron.hoc.HocObject()).copy()
config_fn = './input_files/Parameters.hoc'
neuron.h.xopen(config_fn)

# find all points in radius c_rad from a single point pt in a KDTree kdt
def pt_in_tr (kdt, pt, c_rad):
    warnings.simplefilter('ignore')
    ind, = kdt.query_radius(pt, r = c_rad)
    return ind



gg = Golgi_pop(h)
gg.load_somata(go_32)
gg.add_dendrites()

gp = Granule_pop(h)
gp.load_somata(gr_32)
gp.add_aa_endpoints_fixed()
gp.add_pf_endpoints()

cc = Connect_2D(gg.qpts, gp.qpts_aa, 30)
kdt, q_pts, lax_c, lax_range, lin_in_tree = cc.get_tree_setup()
    

a = pt_in_tr(kdt, q_pts[0], 30)
print (len(a))

1150


# joblib multithreading

In [ ]:
from joblib import Parallel, delayed

gg = Golgi_pop(h)
gg.load_somata(go_4)
gg.add_dendrites()

gp = Granule_pop(h)
gp.load_somata(gr_4)
gp.add_aa_endpoints_fixed()
gp.add_pf_endpoints()

cc = Connect_2D(gg.qpts, gp.qpts_aa, 30)
kdt, q_pts = cc.search_connections()


print ('none')
%timeit a = [pt_in_tr(kdt,pt,30) for pt in q_pts]

print('njobs = 1')
%timeit a1 = (Parallel(n_jobs=1)(delayed(pt_in_tr)(kdt, pt, 30) for pt in q_pts))

print('njobs = 2')
%timeit a1 = (Parallel(n_jobs=2)(delayed(pt_in_tr)(kdt, pt, 30) for pt in q_pts))

print('njobs = 3')
%timeit a1 = (Parallel(n_jobs=3)(delayed(pt_in_tr)(kdt, pt, 30) for pt in q_pts))

print('njobs = 4')
%timeit a1 = (Parallel(n_jobs=4)(delayed(pt_in_tr)(kdt, pt, 30) for pt in q_pts))

print('njobs = 6')
%timeit a1 = (Parallel(n_jobs=6)(delayed(pt_in_tr)(kdt, pt, 30) for pt in q_pts))

In [ ]:
# And now with a loop and a saving file...

n_jobs_a = [-1, 1 , 2, 3, 4, 6]
files = [[go_32, gr_32], [go_16, gr_16], [go_4, gr_4]]
import time
import numpy as np

times = np.zeros((len(files), len(n_jobs_a)))
for fn, [go, gr] in enumerate(files):
                 
    gg = Golgi_pop(h)
    gg.load_somata(go)
    gg.add_dendrites()
    gp = Granule_pop(h)
    gp.load_somata(gr)
    gp.add_aa_endpoints_fixed()
    gp.add_pf_endpoints()
                 
    cc = Connect_2D(gg.qpts, gp.qpts_aa, 30)
    kdt, q_pts = cc.search_connections()
                 
    for njc, nj in enumerate(n_jobs_a):
        start = time.time()
        if nj<0:
            a = [pt_in_tr(kdt,pt,30) for pt in q_pts]
        else:
            a = (Parallel(n_jobs=njc)(delayed(pt_in_tr)(kdt, pt, 30) for pt in q_pts))
        times[fn, njc] = time.time()-start

np.savetxt('times_parallel.txt', times)
            



It seems as if this is not really succesfull, multithreading does increase the computation time. 
This is probably because sklearn KDTree uses Cython (https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/neighbors/kd_tree.pyx), so a GIL (Global Interpreter Lock) is enabled, which prevents more than one native thread active. 

Thus, multithreading seems not to be a useful option, and we should look into multiprocessing.

http://people.duke.edu/~ccc14/sta-663-2016/19C_IPyParallel.html



https://stackoverflow.com/questions/5128072/how-efficient-is-threading-in-python
-> not very efficient due to GIL, have to use multiprocessing (actually, not a complete answer, however:)

https://softwareengineering.stackexchange.com/questions/186889/why-was-python-written-with-the-gil
-> CPython definitely does, and KDTrees from sklearn do use that.

https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python

https://stackoverflow.com/questions/18114285/python-what-are-the-differences-between-the-threading-and-multiprocessing-modul
-> good explanation



## Visualization

In [ ]:
%run BREPpy.py


import matplotlib.pyplot as plt
%matplotlib inline
gg.plot_somata(True, 'ro')
gp.plot_somata(False, 'k.', markersize = 0.1)

In [ ]:
## plot some full cells
gr_pop = gp
go_pop = gg

gr_cells = np.random.randint(0, len(gr_pop.som), 10)
go_cells = np.random.randint(0, len(go_pop.som), 2)


# 3D plot
plt.figure()
ax = plt.gcf().gca(projection='3d')
def plot3d (ax, dat, *args, **kwargs):
    if len(dat.shape) == 1: #if single dot
        dat = np.expand_dims(dat, axis = 0)
    ax.plot(dat[:,0], dat[:,1], dat[:,2], *args, **kwargs)

for c in gr_cells:
    plot3d(ax, gr_pop.aa_dots[c], 'c')
    plot3d(ax, gr_pop.pf_dots[c], 'b')
    plot3d(ax, gr_pop.som[c], 'bo')

for c in go_cells:
    plot3d(ax, go_pop.a_dend[c], 'r')
    plot3d(ax, go_pop.b_dend[c], 'm')
    plot3d(ax, np.expand_dims(go_pop.som[c], axis = 0), 'ro')
    
    
#2D projections
plt.figure(figsize=(15,5))
for i, [j, k, tit] in enumerate([[0,1, 'x-y plane'], [1,2, 'y-z plane'], [0,2, 'x-z plane']]):
    
    plt.subplot(1,3,i+1)
    
    for c in gr_cells:
        plt.plot(gr_pop.aa_dots[c,:,j], gr_pop.aa_dots[c,:,k], 'c')
        plt.plot(gr_pop.pf_dots[c,:,j], gr_pop.pf_dots[c,:,k], 'b')
        plt.plot(gr_pop.som[c,j], gr_pop.som[c,k], 'bo')

    for c in go_cells:
        plt.plot(go_pop.a_dend[c,:,j], go_pop.a_dend[c,:,k], 'r')
        plt.plot(go_pop.b_dend[c,:,j], go_pop.b_dend[c,:,k], 'm')
        plt.plot(go_pop.som[c,j], go_pop.som[c,k], 'ro')
    
    if True:
        if i in [0,2]: plt.xlim([-100, 500])
    plt.title (tit)

In [ ]:
qpt = Query_points(gg.som)

qpt.npts

# Visualization of Golgi cells

For both the apical and the basal dendrites, a parameter GoC_*d_nseg is defined by `int((GoC_ApicalDendL/(GoC_d_lambda*GoC_d_lambda_f)+0.9)/2)*2 + 1 ` (analogous for BasolateralDendL)
Plugging in the values for the other parameters specified, I get that only if the length parameter GoC_**DendL is >= 488.6 (which seems an unlikely case), this parameter is not 1. (Values in the current config file are 166 for apical, 60 for basolateral)
I would like to understand what this parameter does and what it is used for (something cable equationish?)


# Multiprocessing library

In [14]:
import multiprocessing

def worker( num):
    """worker function"""
    a = np.sqrt(num)
    print ('Worker', num, ', result', a)
    return num*a


jobs = []
for i in range(5):
    p = multiprocessing.Process(target=worker, args = (i,)) #note the comma here which makes the integer i iterable.
    jobs.append(p)
    r = p.start()
    
print (r)


Worker 0 , result 0.0
Worker 1 , result 1.0
Worker 2 , result 1.41421356237
Worker 3 , result 1.73205080757
Worker 4 , result 2.0
None


# IPyparallel
http://ipyparallel.readthedocs.io/en/latest/process.html

In [15]:
! ipcluster start -n 4

2017-11-20 12:36:38.574 [IPClusterStart] Removing pid file: /home/ines/.ipython/profile_default/pid/ipcluster.pid
2017-11-20 12:36:38.584 [IPClusterStart] Starting ipcluster with [daemon=False]
2017-11-20 12:36:38.585 [IPClusterStart] Creating pid file: /home/ines/.ipython/profile_default/pid/ipcluster.pid
2017-11-20 12:36:38.585 [IPClusterStart] Starting Controller with LocalControllerLauncher
2017-11-20 12:36:39.589 [IPClusterStart] Starting 4 Engines with LocalEngineSetLauncher
^C
2017-11-20 12:37:07.146 [IPClusterStart] ERROR | IPython cluster: stopping
2017-11-20 12:37:07.166 [IPClusterStart] Stopping Engines...


In [ ]:
! ipython profile create --parallel --profile=myprofile

in the directory
~/.ipython/
you can find the different directories which would now have to be populated


In [ ]:
import ipyparallel as ipp
c = ipp.Client()
print (c.ids)
c[:].apply_sync(lambda: 'Hello, World')

In [ ]:
ipp.Client?

In [ ]:
from IPython import parallel
rc = parallel.Client()

rc.block = True # As I get it, this parameter says whether or not to wait until some job is done before return.
                # If false, this will lead to possibly asynchronous return, but if True it might be slightly slower

#simble process
def mul(a,b):
    return a*b


def summary():
    """summarize some info about this process"""
    import os
    import socket
    import sys
    print('cwd:     ', os.getcwd())
    print('Python:  ', sys.version)
    print('hostname:', socket.gethostname())
    print('pid:     ', os.getpid())
    return {
        'cwd': os.getcwd(),
        'Python': sys.version,
        'hostname': socket.gethostname(),
        'pid': os.getpid(),
    }

In [ ]:
#Local execution
m = mul(5,6)
_=summary()
print (m)

In [ ]:
#Now we work remotely
r = rc[0].apply(mul, 5, [6,7,8,9])
rc[0].apply(summary)

print (r)

In [ ]:
#And now in parallel.
rc[:].apply(mul, 5, 6)
rc[:].apply(summary)

In [ ]:
map(mul, range(1,10), range(2,11))
view = rc.load_balanced_view()
view.map(mul, range(1,10), range(2,11))

## Applying this to our own project

In [44]:
%run BREPpy.py



p1 = './example_simulation/coordinates_input/'
p2 = './example_simulation/coordinates_input/subsampled/'

go_ori = p1+'GoCcoordinates.dat'
gr_ori = p1+'GCcoordinates.dat'
go_64 = p2+'GoCcoordinates_64.dat'
gr_64 = p2+'GCcoordinates_64.dat'
go_16 = p2+'GoCcoordinates_16.dat'
gr_16 = p2+'GCcoordinates_16.dat'
go_4 = p2+'GoCcoordinates_4.dat'
gr_4 = p2+'GCcoordinates_4.dat' 

# read in parameter file
empty_hoc = dir(neuron.hoc.HocObject()).copy()
config_fn = './input_files/Parameters.hoc'
neuron.h.xopen(config_fn)

# find a single point in a KDTree
def pts_in_tr (kdt, pt, c_rad):
    import numpy as np
    import warnings
    if sum(np.array(pt.shape)>1)==1: pt = [pt]
    res = []
    for p in pt: 
        warnings.simplefilter('ignore')
        ind, = kdt.query_radius(p, r = c_rad)
        res.append(ind,)
    return res

def pt_in_tr (kdt, pt, c_rad):
    import warnings
    warnings.simplefilter('ignore')
    res, = kdt.query_radius(pt, r = c_rad)
    return res

#pt_in_tr (kdt, pt, c_)


gg = Golgi_pop(h)
gg.load_somata(go_64)
gg.add_dendrites()

gp = Granule_pop(h)
gp.load_somata(gr_64)
gp.add_aa_endpoints_fixed()
gp.add_pf_endpoints()

c_rad = 15
cc = Connect_2D(gg.qpts, gp.qpts_aa, c_rad)
kdt, q_pts, lax_c, lax_range, lin_in_tree = cc.get_tree_setup()


In [37]:
ts1 = time.time()
r1, l_r1 = cc.find_connections()
ts2 = time.time()

In [6]:
import ipyparallel as ipp
rc = parallel.Client()
import time

In [45]:
rc = ipp.Client()
dv = rc[:]
lv = rc.load_balanced_view()

t1 = time.time()
dv.block = True
with dv.sync_imports(): # note: import as does not work as only import part works, not assignment. Also, you will have to store the file in a directory reachable from the PYTHONPATH
    import parallel_util_test 
dv.push(dict(kdt = kdt, q_pts = q_pts))

lam_qpt = lambda pt: parallel_util_test.pt_in_tr(kdt, pt, 15)
dv.block = False
t2 = time.time()
res = list(lv.map (lam_qpt, q_pts))
t3 = time.time()




importing parallel_util_test on engine(s)


In [46]:
print (t2-t1)
print (t3-t2)
print (ts2-ts1)
print (len(res))

print (res[1])

-0.31743836402893066
-67.44466352462769
-1.9915156364440918
2220
[11493  5633  7415  8072  4947  2601  4174 10172  3822 10943  7030  6322
 12358 12393  3985  5983 12797  3548  3562  9528  3266  5494   143  6066
  5928  7710  4296  5960  1108  5401  3803  1884  5830  6709  8717  1051
  8396  4419  2174  5676  7006 12305  6221  7070  4956  3515  7072  9950
  5826  4227  4620  5202  9817  4949   435  4151  2105  4664  8473 12869
  8880 12678  1929 11373  3637  8761  6631  4726  9664  6556  5149  9986
  9233  5653  6757  1618  5705  1096 10842  2537  4433 12074  2367  5901
   513  6909 12035   907  6925  1728  5984  8682  8691    34  6538  5627
  4106 12773  6962 10122   806  6358 12874  6795  6672 10314  2029  7791
  5545 10204  7241 12591 13022  5143 13096 10014  7298  3300 11252  7941
  9047    62 11557  5534  7894  2907  1031  6410  8855  4291  8411 11691
  5638  2264 12067  1246 10585  7926 12029 10712  9689  6364 10527  1008
  7329 13045  2049  6650  5773  7521  2321  7262  4867  846

In [41]:
print (ts1 - ts2)

-1.9915156364440918


In [26]:
#import parallel_util_test
import sys
import os
sys.path.append('~/home/ines/Desktop/LabRot_OIST/pybrep')

print (sys.meta_path)

[<class '_frozen_importlib.BuiltinImporter'>, <class '_frozen_importlib.FrozenImporter'>, <class '_frozen_importlib_external.PathFinder'>, <six._SixMetaPathImporter object at 0x7fddfedcb908>, <pkg_resources.extern.VendorImporter object at 0x7fddfba8aac8>, <pkg_resources._vendor.six._SixMetaPathImporter object at 0x7fddfb5abda0>, <matplotlib.externals.six._SixMetaPathImporter object at 0x7fddd08ab6d8>, <IPython.utils.shimmodule.ShimImporter object at 0x7fddac2f4128>]


In [27]:
! echo $PYTHONPATH

/home/ines/.local/lib/auto/07p/python/::/usr/local/nrn/lib/python/


In [ ]:
print (t2-t1)

In [ ]:
a = pt_in_tr(kdt, q_pts[0], 15)
#print (a)

r = rc[0].apply(mul, 5, [6,7,8,9])
rc[0].apply(summary)

start = time.time()
r = [rc[i%4].apply(pts_in_tr, kdt, q_pts[i,:], 15) for i in range(len(q_pts))]



In [ ]:
n_jobs = 4

idx = np.linspace(0, len(q_pts)-1, n_jobs+1).astype('int')
import time
start = time.time()
rr = [rc[i].apply(pts_in_tr, kdt, q_pts[idx[i]:idx[i+1],:], 15) for i in range(n_jobs)]

stop = time.time()-start
print (stop)

In [ ]:
r = []
#for sr in map(pt_in_tr, [kdt], [q_pts[i,:] for i in range(len(q_pts))], [15]):
#    r.append(sr)
#print (qpl)
#print (qpl)


def pt_in_tr (kdt, pt, c_rad):
    #print (pt)
    #print (c_rad)
    warnings.simplefilter('ignore')
    ind, = kdt.query_radius(pt, r = c_rad)
    return ind

r = [*map(lambda pt: pt_in_tr(kdt, pt, 15), q_pts)]

In [ ]:
dview = rc[:]

#%timeit r2 = dview.map_sync(lambda pt: pt_in_tr(kdt, pt, 15), q_pts) 

# Parallelization unrelated insights found while looking at tutorials
https://github.com/minrk/IPython-parallel-tutorial/blob/42ae138529d652d7f18925c619dd305dde579a2f/Background.ipynb

In [ ]:
files = !ls
#print (files)
print (repr(files))
# repr gives the canonical string representation of an object


In [ ]:
#some more magics
%lsmagic #gives you the different magics!

In [ ]:
def inner(x):
    return 1/x

def outer(y):
    return inner(1-y)

z = 2
outer(-z + 3)
%debug

In [ ]:
%%latex
#here comes cell magic (must really be in the first line, or it won't work)
\begin{eqnarray}
\nabla \times \vec{\mathbf{B}} &
    - \frac{1}{c} \frac{\partial\vec{\mathbf{E}}}{\partial t} 
    & = & \frac{4 \pi}{c} \vec{\mathbf{j}} \\
\nabla \cdot \vec{\mathbf{E}} && = & 4 \pi \rho \\
\nabla \times \vec{\mathbf{E}} &
    + \frac{1}{c} \frac{\partial\vec{\mathbf{B}}}{\partial t}
    & = & \vec{\mathbf{0}} \\
\nabla \cdot \vec{\mathbf{B}} && = & 0 
\end{eqnarray}

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jn
x = np.linspace(0,10)
for n in range(5):
    plt.plot(x, jn(n,x))


In [ ]:
from IPython.display import display
from IPython.html.widgets import interact
import sympy
sympy.init_printing(use_latex='mathjax')
x = sympy.symbols('x')
expr = x**10-1
expr
sympy.factor(expr)


In [ ]:
@interact
def factorit(n=10):
    display(sympy.factor(x**n-1))

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('-Lw_smO-CjQ')

In [ ]:
name = "pyB"
!head -n 100 {name}.ipynb | pygmentize -l json

In [ ]:
%connect_info #-> gives the number and the address of the kernel you use
%qtconsole #-> opens the console with iqdb
%debug #-> starts a iqdb pocess inline
#-> here, if you print debug, you will jump to the last error. Leave with q
